In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# single cell
import scanpy.api as sc
from anndata import AnnData

# etc
%load_ext rpy2.ipython

### Regress out read depth per experiment

In [ ]:
wd = 'lung_snATAC/'

adatas = {}
samples = ['D062','D088','D110','D150','D032','D032-2','D046','D139','D122','D175','D231']

# dictionary naming 5kb windows genome-wide based on overlap with gencode v19 gene TSS
promoters = pd.read_csv(os.path.join('references', 'gencode.v19.5kb_windows.promoter_names.txt.gz'), sep='\t', header=None, index_col=0, names=['prom'])
promoter_names = promoters['prom'].to_dict() 

# cells from low quality and doublet clusters were identified through iterative clustering
low_frip = open(os.path.join(wd, 'lung_snATAC.merged_samples.lowfrip')).read().splitlines()
low_reads = open(os.path.join(wd, 'lung_snATAC.merged_samples.lowreads')).read().splitlines()
doublets = open(os.path.join(wd, 'lung_snATAC.merged_samples.doublets')).read().splitlines()

qc_metrics = pd.read_csv(os.path.join(wd, 'lung_snATAC.merged_samples.qc_metrics.txt'), sep='\t', header=0, index_col=0)
hvw = open(os.path.join(wd,'lung_snATAC.merged_samples.hvw')).read().splitlines()

for sample in samples:
    sp = scipy.io.mmread(os.path.join(wd, sample, '{}.mtx.gz'.format(sample))).tocsr()
    regions = open(os.path.join(wd, sample, '{}.regions'.format(sample))).read().splitlines()
    barcodes = open(os.path.join(wd, sample, '{}.barcodes'.format(sample))).read().splitlines()
    adatas[sample] = AnnData(sp, {'obs_names':barcodes}, {'var_names':regions})
    adatas[sample].var.index = [promoter_names[b] if b in promoter_names else b for b in adatas[sample].var.index]
    adatas[sample].var_names_make_unique(join='.')
    
    adatas[sample] = adatas[sample][~adatas[sample].obs.index.isin(low_frip + low_reads + doublets),:].copy()
    adatas[sample].obs = adatas[sample].obs.join(qc_metrics, how='inner')
    adatas[sample].obs['experiment'] = [i.split('_')[0] for i in adatas[sample].obs.index]
    raw = adatas[sample].copy()
    
    sc.pp.normalize_per_cell(adatas[sample], counts_per_cell_after=1e4)
    adatas[sample] = adatas[sample][:, adatas[sample].var.index.isin(hvgs)]
    sc.pp.log1p(adatas[sample])
    adatas[sample].obs['log_usable_counts'] = np.log(raw[:, raw.var.index.isin(hvgs)].X.sum(axis=1).A1)
    sc.pp.regress_out(adatas[sample], ['log_usable_counts'])
    adatas[sample].write(os.path.join(wd, '{}.norm.h5ad'.format(sample)))
    
    sc.pp.normalize_per_cell(raw, counts_per_cell_after=1e4)
    sc.pp.log1p(raw)
    raw.write(os.path.join(wd, '{}.raw.h5ad'.format(sample)))


### Merge files from all samples

In [ ]:
adatas = {}
adatas_raw = {}
samples = ['D062','D088','D150','D110', 'D032','D032-2','D046','D139','D122','D175','D231']

for sample in samples:
    adatas[sample] = sc.read_h5ad(os.path.join(wd, '{}.norm.h5ad'.format(sample)))
    adatas_raw[sample] = sc.read_h5ad(os.path.join(wd, '{}.raw.h5ad'.format(sample)))
    
adata_norm = AnnData.concatenate(adatas['D062'], adatas['D088'], adatas['D110'], adatas['D150'],
                                 adatas['D032'], adatas['D032-2'], adatas['D046'], adatas['D139'],
                                 adatas['D122'], adatas['D175'], adatas['D231'],
                                 batch_key='norm', index_unique=None)
adata_norm_raw = AnnData.concatenate(adatas_raw['D062'], adatas_raw['D088'], adatas_raw['D110'], adatas_raw['D150'],
                                     adatas_raw['D032'], adatas_raw['D032-2'], adatas_raw['D046'], adatas_raw['D139'],
                                     adatas_raw['D122'], adatas_raw['D175'], adatas_raw['D231'],
                                     batch_key='norm', index_unique=None)
adata_norm.raw = adata_norm_raw.copy()

sc.pp.scale(adata_norm)
sc.tl.pca(adata_norm, zero_center=True, svd_solver='arpack', random_state=0)
pc = pd.DataFrame(adata_norm.obsm['X_pca'], columns=['PC{}'.format(i) for i in range(1,51)], index=adata_norm.obs.index)
metadata = pd.read_csv(os.path.join(wd, 'lung_snATAC.merged_samples.metadata.txt'), sep='\t', header=0, index_col=0)
metadata = metadata.loc[pc.index]

### Run Harmony (rpy2) to correct for batch effects

In [ ]:
%%R -i pc -i metadata -o harmonized
library(harmony)
library(magrittr)

# run Harmony on the PCs
harmonized <- HarmonyMatrix(pc, metadata, c('experiment','sex'), do_pca=FALSE)
harmonized <- data.frame(harmonized)

### Plot cluster based on corrected components

In [ ]:
adata_norm.obsm['X_pca'] = harmonized.values
sc.pp.neighbors(adata_norm, n_neighbors=30, method='umap', metric='cosine', random_state=0, n_pcs=50)
sc.tl.leiden(adata_norm, resolution=1.5, random_state=0)
sc.tl.umap(adata_norm, min_dist=0.3, random_state=0)

sc.settings.set_figure_params(dpi=100)
sc.pl.umap(adata_norm, color=['leiden'], size=1, alpha=.5)
sc.pl.umap(adata_norm, color=['experiment'], size=1, alpha=.5)

# plot quality metrics
sc.pl.umap(adata_norm, color=['log_usable_counts'], size=1, color_map='Blues')
sc.pl.umap(adata_norm, color=['frac_reads_in_peaks','frac_reads_in_promoters','frac_promoters_used'], cmap='Reds', size=1, legend_loc='on data')

# 5kb windows overlapping marker promoters    
sc.pl.umap(adata_norm, color=['SLC11A1','CD247','MS4A1'], size=9, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['SFTPC','EDN3','MCIDAS'], size=9, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['VWF','GBP4','PROX1'], size=9, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['WNT2','MYOCD','GJC1'], size=9, color_map='Blues', frameon=True, use_raw=True)


In [ ]:
# More 5kb windows overlapping marker promoters    
sc.pl.umap(adata_norm, color=['C1QA','C1QB','C1QC'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['CD3E','CD8A','BCL11B'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['NCR1','SH2D1B','IFNG'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['CD79A','CD19','MS4A1'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['SERTM1','EDN3','CTNND2'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['SFTPB','SFTPA1','ACOXL'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['SOX2','TEKT4','MCIDAS'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['CLEC14A','EGFL7','CDH5'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['VWF','PTPRB','CD93'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['ART1','APOB','EML1'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['PROX1','PDE1A','HOXD8'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['WNT2','MEOX2','SCN7A'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['DNAI2','GJC1','FOXS1'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['DES','MYOCD','PRELP'], size=4, color_map='Blues', frameon=True, use_raw=True)
sc.pl.umap(adata_norm, color=['CD34','MFAP5','FBLN2'], size=4, color_map='Blues', frameon=True, use_raw=True)


### Subclustering at high resolution to identify potential doublet subclusters

In [ ]:
subset_cluster = ['0']
sc.tl.louvain(adata_norm, restrict_to=('leiden',subset_cluster), resolution=3, random_state=0, key_added='subset')
sc.pl.umap(adata_norm, color=['subset'], size=9)

fig, ax1 = plt.subplots(1,1,figsize=(5,5))
subset = adata_norm.obs.join(pd.DataFrame(adata_norm.obsm['X_umap'], index=adata_norm.obs.index, columns=['UMAP1','UMAP2']), how='inner')
subset = subset.loc[subset['leiden'].isin(subset_cluster)]
for s in sorted(set(subset['subset'])):
    ax1.scatter(subset.loc[subset['subset']==s, 'UMAP1'], subset.loc[subset['subset']==s, 'UMAP2'], alpha=1, s=4, label=s)
ax1.legend(markerscale=3)
plt.show()

# plot qc metrics including subclusters
for qc_metric in ['log10_usable_counts', 'frac_reads_in_peaks', 'frac_promoters_used']:
    fig, ax1 = plt.subplots(1,1,figsize=(7,5))
    sns.boxplot(x='subset', y=qc_metric, data=adata_norm.obs, ax=ax1)
    ax1.axhline(adata_norm.obs[qc_metric].median(), color='black', ls='dotted')
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)
    plt.show()

# check marker promoters for potential doublet subclusters
sc.pl.dotplot(adata_norm, ['SLC11A1','CD247','MS4A1','SFTPC','EDN3','MCIDAS','VWF','GBP4','PROX1','WNT2','MYOCD','GJC1'],
              standard_scale='var', groupby='subset', dendrogram=False, use_raw=True)
    
adata_norm.obs.loc[adata_norm.obs['subset'].isin(['0,28'])].to_csv(os.path.join(wd, '{}.alveolar_type_2.doublets'.format(sample_name)), header=False, columns=[])